In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
! kaggle datasets download imneonizer/normal-vs-camouflage-clothes

 99% 1.80G/1.82G [00:10<00:00, 261MB/s]
100% 1.82G/1.82G [00:10<00:00, 185MB/s]


In [6]:
from zipfile import ZipFile
from tqdm import tqdm

with ZipFile('/content/normal-vs-camouflage-clothes.zip', 'r') as zip_file:
    file_list = zip_file.namelist()
    for file_name in tqdm(file_list):
        zip_file.extract(file_name, '/content')


100%|██████████| 15898/15898 [00:16<00:00, 945.23it/s] 


In [10]:
import os
ORIG_INPUT_DATASET = "/content/8k_normal_vs_camouflage_clothes_images"
BASE_PATH = "camo_not_camo"

TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

In [7]:
TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.1
CLASSES = ["camouflage_clothes", "normal_clothes"]

In [8]:
INIT_LR = 1e-4
BS = 32
NUM_EPOCHS = 20
MODEL_PATH = "military_camo_detector.model"

In [9]:
from imutils import paths
import random
import shutil

In [11]:
imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)
# computing training and testing split
i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]
# part of the training data for validation
i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]
# defining the datasets
datasets = [
	("training", trainPaths, TRAIN_PATH),
	("validation", valPaths, VAL_PATH),
	("testing", testPaths, TEST_PATH)
]

In [13]:
os.getcwd() 

'/content'

In [16]:
for (dType, imagePaths, baseOutput) in datasets:
	print("[INFO] building '{}' split".format(dType))
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)
	for inputPath in imagePaths:
		filename = inputPath.split(os.path.sep)[-1]
		label = inputPath.split(os.path.sep)[-2]
		labelPath = os.path.sep.join([baseOutput, label])
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating camo_not_camo/training' directory
[INFO] 'creating camo_not_camo/training/camouflage_clothes' directory
[INFO] 'creating camo_not_camo/training/normal_clothes' directory
[INFO] building 'validation' split
[INFO] 'creating camo_not_camo/validation' directory
[INFO] 'creating camo_not_camo/validation/normal_clothes' directory
[INFO] 'creating camo_not_camo/validation/camouflage_clothes' directory
[INFO] building 'testing' split
[INFO] 'creating camo_not_camo/testing' directory
[INFO] 'creating camo_not_camo/testing/camouflage_clothes' directory
[INFO] 'creating camo_not_camo/testing/normal_clothes' directory


In [18]:
import matplotlib
matplotlib.use("Agg")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [24]:
"""ap = argparse.ArgumentParser()
ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output loss/accuracy plot")
args = vars(ap.parse_args())"""
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))
print("train ",totalTrain, " val ",totalVal," test ",totalTest)

train  10731  val  1192  test  3975


In [25]:
trainAug = ImageDataGenerator(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")
valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [28]:
# training generator
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
# validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
# testing generator
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [29]:
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

[INFO] preparing model...
94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
print("[INFO] training model...")
H = model.fit(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	epochs=NUM_EPOCHS)

[INFO] training model...
Epoch 1/20
335/335 [==============================] - 330s 973ms/step - loss: 0.1943 - accuracy: 0.9298 - val_loss: 0.0861 - val_accuracy: 0.9764
Epoch 2/20
335/335 [==============================] - 321s 957ms/step - loss: 0.0914 - accuracy: 0.9707 - val_loss: 0.0689 - val_accuracy: 0.9755
Epoch 3/20
335/335 [==============================] - 319s 953ms/step - loss: 0.0697 - accuracy: 0.9783 - val_loss: 0.0654 - val_accuracy: 0.9780
Epoch 4/20
335/335 [==============================] - 319s 953ms/step - loss: 0.0627 - accuracy: 0.9794 - val_loss: 0.0803 - val_accuracy: 0.9679
Epoch 5/20
335/335 [==============================] - 321s 958ms/step - loss: 0.0540 - accuracy: 0.9817 - val_loss: 0.0596 - val_accuracy: 0.9797
Epoch 6/20
335/335 [==============================] - 319s 952ms/step - loss: 0.0480 - accuracy: 0.9842 - val_loss: 0.0590 - val_accuracy: 0.9814
Epoch 7/20
335/335 [==============================] - 319s 953ms/step - loss: 0.0511 - accuracy: 0.

In [ ]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BS) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))

print("[INFO] saving model...")
model.save(MODEL_PATH, save_format="h5")

In [ ]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])